In [1]:
!pip install transformers
!pip install datasets
!pip install Tensorflow
!pip install Flax

  Using cached ml_dtypes-0.2.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (20 kB)
Using cached ml_dtypes-0.2.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.0 MB)
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.3.2
    Uninstalling ml-dtypes-0.3.2:
      Successfully uninstalled ml-dtypes-0.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorstore 0.1.52 requires ml-dtypes>=0.3.1, but you have ml-dtypes 0.2.0 which is incompatible.
  Using cached ml_dtypes-0.3.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (20 kB)
Using cached ml_dtypes-0.3.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.2 MB)
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtyp

In [2]:
from collections import defaultdict
from tqdm import tqdm
import requests

from transformers import GPT2Tokenizer
from datasets import load_dataset, Dataset

import os
import s3fs

In [3]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium')

In [4]:
def any_keyword_in_string(string, keywords):
    for keyword in keywords:
        if keyword in string:
            return True
    return False

def filter_streaming_dataset(dataset, filters):
    filtered_dict = defaultdict(list)
    total = 0
    for sample in tqdm(iter(dataset)):
        total += 1
        if any_keyword_in_string(sample['text'], filters):
            for k, v in sample.items():
                filtered_dict[k].append(v)
    print(f"{len(filtered_dict['text'])/total:.2%} of data after filtering.")
    return Dataset.from_dict(filtered_dict)

In [5]:
from datasets import load_dataset, DatasetDict

#Defining filters and the purpose of our dataset:
split = "train[:1%]"
filters = []

#Loading the dataset with the filter intended (or not):
ds_train = load_dataset('wikipedia', '20220301.en', split=split)
#filtered_data = filter_streaming_dataset(ds_train, filters)

/opt/mamba/lib/python3.11/site-packages/datasets/load.py:1429: FutureWarning: The repository for wikipedia contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/wikipedia
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


ValueError: Unrecognized instruction format: train[:0.5%]

In [ ]:
#Tokenizing:

from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium')
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
context_length = 128

outputs = tokenizer(ds_train["text"][:2], truncation=True, max_length=context_length, padding="max_length")

#Now adapting the code from the article (https://huggingface.co/learn/nlp-course/chapter7/6?fw=pt) so it fits the data set we are using ():

def tokenize(element):
    outputs = tokenizer(element["text"],truncation=True,max_length=context_length,return_overflowing_tokens=True,return_length=True)
    input_batch = []
    for length, input_ids in zip(outputs["length"], outputs["input_ids"]):
        if length == context_length:
            input_batch.append(input_ids)
    return {"input_ids": input_batch}

tokenized_datasets = ds_train.map(tokenize, batched=True, remove_columns=ds_train.column_names)


In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AutoConfig

config = AutoConfig.from_pretrained("gpt2",vocab_size=len(tokenizer),n_ctx=context_length,bos_token_id=tokenizer.bos_token_id,eos_token_id=tokenizer.eos_token_id)

In [ ]:
!pip install torch==2.1.2+cpu -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
from transformers import AutoTokenizer, GPT2LMHeadModel, AutoConfig
import torch

model = GPT2LMHeadModel(config)
model_size = sum(t.numel() for t in model.parameters())
#print(f"GPT-2 size: {model_size/1000**2:.1f}M parameters")

In [ ]:
!pip install TensorRT

In [ ]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

In [ ]:
out = data_collator([tokenized_datasets["train"][i] for i in range(5)])
for key in out:
    print(f"{key} shape: {out[key].shape}")